## Weavechain integrity proofs on ICP

In this demo notebook we will showcase storing integrity proofs on ICP:

- we will create a table in a data collection
- mark some fields as personal information that is not to be shared
- write few records
- read data locally (and be able to see all fields)
- read hashes stored on ICP
- verify integrity proofs stored on ICP
   
A Weavechain node is installed and preconfigured to support this scenario

### 1. Create an API session

In [24]:
import pandas as pd

from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

WEAVE_CONFIG = "config/demo_client_remote_rolodex.config"
nodeApi, session = connect_weave_api(WEAVE_CONFIG)

scope = "rolodex"
table = "directory"

{"res":"ok","data":"pong 1697703766258"}


### 2. Create a local table

- we drop the existing table if already existing and re-create it from scratch
- a weavechain node can also connect to existing tables, reading their structure, but in this case we create it via the API

In [25]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "ts": { "type": "LONG" },
        "writer": { "type": "STRING" },
        "sig": { "type": "STRING" },
        "name_nickname": { "type": "STRING" },
        "name_last": { "type": "STRING" },
        "name_first": { "type": "STRING" },
        "birthday": { "type": "STRING" },
        "email_personal": { "type": "STRING" },
        "phone_number": { "type": "STRING" },
        "address_country": { "type": "STRING" },
        "address_summary": { "type": "STRING" },
        "address_timezone": { "type": "STRING" },
        "linkedin_url": { "type": "STRING" },
        "discord_username": { "type": "STRING" },
        "telegram_username": { "type": "STRING" },
        "ethereum_wallet_address": { "type": "STRING" }
    }, 
    "idColumnIndex": 0, 
    "timestampColumnIndex": 1, 
    "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
    "signatureColumnIndex": 3,
    "isLocal": False,
    "applyReadTransformations": True
}

nodeApi.dropTable(session, scope, table).get()
reply = nodeApi.createTable(session, scope, table, CreateOptions(False, False, layout)).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'icprolodex', 'account': 'icprolodex', 'scope': 'rolodex', 'table': 'directory'}}


### 3. Mark some fields for erasure

- the purpose is to protect certain fields when shared

In [26]:
reply = nodeApi.getTableDefinition(session, scope, table).get()
#print(reply)
layout = json.loads(reply["data"])["layout"]

newLayout = layout.copy()
del newLayout["layout"]
del newLayout["indexes"]
del newLayout["columnNames"]
newLayout["columns"] = { i["columnName"]: i for i in layout["columns"]}

newLayout["columns"]["phone_number"]["readTransform"] = "ERASURE"
newLayout["columns"]["address_summary"]["readTransform"] = "ERASURE"
newLayout["columns"]["ethereum_wallet_address"]["readTransform"] = "ERASURE"
newLayout["columns"]["birthday"]["readTransform"] = "ERASURE"

reply = nodeApi.updateLayout(session, scope, table, newLayout).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'UPDATE_LAYOUT', 'organization': 'icprolodex', 'account': 'icprolodex', 'scope': 'rolodex', 'table': 'directory'}}


### 4. Write few records in the local storage

In [27]:
records = Records(table, [ 
    [ None, None, None, None, 'Nickname', 'Last Name', 'First name', '1980-01-01', 'email@gmail.com', '+40712345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#1234', '@telegram', '0xwallet' ],
    [ None, None, None, None, 'jdoe', 'John', 'Doe', '1990-05-01', 'johndoe@gmail.com', '+44712345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#jdoe', '@jdoe', '0xwallet' ],
    [ None, None, None, None, 'jndoe', 'Jane', 'Doe', '2001-03-01', 'janedoe@gmail.com', '+112345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#1234', '@telegram', '0xwallet' ],
    [ None, None, None, None, 'foobar', 'Foo', 'Bar', '1989-12-01', 'fbar@gmail.com', '+122345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#1234', '@telegram', '0xwallet' ]
])
reply = nodeApi.write(session, scope, records, WRITE_DEFAULT).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'icprolodex', 'account': 'icprolodex', 'scope': 'rolodex', 'table': 'directory'}, 'data': 'weaved8xLnTMp5B5GtJwDQvc1u7K4fwPc2ry7iDieyCdJRHcG,5U6dfEWTjJcyA1yxtbDsZ4yu5NPpfd/pl0dNoCi+I6A=,YmcLrYANb6g61X689EUSun4QTNgZ9fFcYJYCnjJmPPpAHWHZU2x48u3yV7vysMhfVNsaWzNjchFMHo5bJeCK5aX', 'ids': '1 2 3 4'}


### 5. Read the local record, from the local storage

- since we read with the owner key and from the local node, we expect the records to have all fields visible

In [28]:
scope = "rolodex"
table = "directory"

reply = nodeApi.read(session, scope, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])

df.head()

,id,ts,writer,sig,name_nickname,name_last,name_first,birthday,email_personal,phone_number,address_country,address_summary,address_timezone,linkedin_url,discord_username,telegram_username,ethereum_wallet_address
0,1,1697703781148,weaved8xLnTMp5B5GtJwDQvc1u7K4fwPc2ry7iDieyCdJRHcG,"[{""sig"":{""recordsHash"":""5U6dfEWTjJcyA1yxtbDsZ4...",Nickname,Last Name,First name,1980-01-01,email@gmail.com,+40712345678,US,Secret,EST,https://www.linkedin.com/in/linkedin/,discord#1234,@telegram,0xwallet
1,2,1697703781148,weaved8xLnTMp5B5GtJwDQvc1u7K4fwPc2ry7iDieyCdJRHcG,"[{""sig"":{""recordsHash"":""5U6dfEWTjJcyA1yxtbDsZ4...",jdoe,John,Doe,1990-05-01,johndoe@gmail.com,+44712345678,US,Secret,EST,https://www.linkedin.com/in/linkedin/,discord#jdoe,@jdoe,0xwallet
2,3,1697703781148,weaved8xLnTMp5B5GtJwDQvc1u7K4fwPc2ry7iDieyCdJRHcG,"[{""sig"":{""recordsHash"":""5U6dfEWTjJcyA1yxtbDsZ4...",jndoe,Jane,Doe,2001-03-01,janedoe@gmail.com,+112345678,US,Secret,EST,https://www.linkedin.com/in/linkedin/,discord#1234,@telegram,0xwallet
3,4,1697703781148,weaved8xLnTMp5B5GtJwDQvc1u7K4fwPc2ry7iDieyCdJRHcG,"[{""sig"":{""recordsHash"":""5U6dfEWTjJcyA1yxtbDsZ4...",foobar,Foo,Bar,1989-12-01,fbar@gmail.com,+122345678,US,Secret,EST,https://www.linkedin.com/in/linkedin/,discord#1234,@telegram,0xwallet


#### Check row integrity proof from writer

In [35]:
display(json.loads(df.iloc[-1]["sig"]))

[{'sig': {'recordsHash': '5U6dfEWTjJcyA1yxtbDsZ4yu5NPpfd/pl0dNoCi+I6A=',
   'count': '1',
   'pubKey': 'weaved8xLnTMp5B5GtJwDQvc1u7K4fwPc2ry7iDieyCdJRHcG',
   'sig': '4rs3uHoksFWVaYCDKA78smAHiK4SrLiDxvbEV5qB3NDdTffXuNv2griLLC96PYNzYbfYTTDKvvdh5L88x51EWSSA'}}]

### 6. Read hashes

In [36]:
reply = nodeApi.hashes(session, scope, table, None, READ_DEFAULT_NO_CHAIN).get()
dh = pd.DataFrame([ [x, reply["data"][x]] for x in list(reply["data"]) ], columns=["ID", "Hash"])

display(dh)

,ID,Hash
0,1,NNBDoDj64aDv21DJQ+K6xFG8xQ1Ry6TN9zhAR8MI6p8=


### 7. Read merkle root for the table

In [37]:
from binarytree import Node

def showtree(tree):
    prev = None
    root = None
    lvl = 1
    for l in tree.split(";"):
        level = l.split(",")
        print(lvl, level)
        lvl += 1
        nodes = []
        for i in range(len(level)):
            pidx = int(i / 2)
            node = Node(level[i][:3] + ".." + level[i][-3:])
            if root is None:
                root = node
            nodes.append(node)
            if prev is not None:
                parent = prev[pidx]
                if parent.left is None:
                    parent.left = node
                else:
                    parent.right = node
        prev = nodes
    
    print(root)

salt = "salt1234" # Same salt used for records hashes, this can be improved to have different salts for *each distinct writer*
digest = "SHA-256"

filter = Filter(None, None, None, None, [ "name_nickname","name_last","name_first","birthday","email_personal","phone_number","address_country","address_summary","address_timezone","linkedin_url","discord_username","telegram_username","ethereum_wallet_address" ])
reply = nodeApi.merkleTree(session, scope, table, filter, salt, digest, READ_DEFAULT_NO_CHAIN).get()
tree = reply["data"]["tree"]
rootHash = reply["data"]["rootHash"]
ts = reply["data"]["timestamp"]
rootHashSignature = reply["data"]["signature"]

print("Generated at", ts)
print("Root Hash", rootHash)
print("Signature", rootHashSignature)
print("")
showtree(tree)
# We've built a Merkle Tree at a specific time, signed by the node that created it.
# The Merkle Leaves are salted hashes of the data in the table

Generated at 1697703841031
Root Hash EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5
Signature YBSscPH9YnKYx5re1399dKo6cPxHGoKfmq3Mnn6JAG5XLwx3uAwTumSzKE2BneyZyBeML2vjmZSpcU7VcQ14yVt

1 ['EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5']
2 ['4ZzDmf2gyKgxh9i8kbULc95p8HQSWsxXJh1waq9o4bhy', '5JGTsLpou5VTcepwwmvEAR2i6CULnMUPEgwGa3e1hrCg']
3 ['26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb', '4eWYP6nWEH6WVS6Yw7JD3ENgCmiLCfz5YUEPKCTV1PMH', '9Xea1VRN5Jm1s3yLmFczCFAgVSF7v4fCwF3MGKTYkFz', '3QFodfQQiD5e4vDZ72eh2jFVqtsXPsQXYjxswsaKXECZ']

              _____________Enw..pX5____________
             /                                 \
     ____4Zz..bhy___                     ____5JG..rCg___
    /               \                   /               \
26c..HLb          4eW..PMH          9Xe..kFz          3QF..ECZ



### Check merkle root stored on ICP

In [38]:
reply = nodeApi.rootHash(session, scope, table).get()
data = reply["data"]
display(data)

{'signature': '3P2Z3rjBWveUM8eKLKDbD6fLzTQGoQr2g8xzn6U8kjzgEY49NXyD2WCXaJ6DvM85DH9A3YTcbrCbLYFhFEKEEN8k',
 'rootHash': 'EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5',
 'timestamp': '1697703843484'}

### Verify the root hash against ICP

In [39]:
reply = nodeApi.rootHash(session, scope, table).get()
data = reply["data"]
display(data)

chainRootHash = data["rootHash"]
print("\nMatching:", rootHash == chainRootHash) # data hashes are salted. The salt needs to match what's configured on the network to have a match

signingKey = nodeApi.sigKey().get()["data"]
print("Node Public Key:", signingKey)

toSign = rootHash + " " + ts
#print(rootHash)
check = nodeApi.verifySignature(rootHashSignature, toSign)
print("Check signature from merkleTree() call:", check)

toSign = data["rootHash"] + " " + data["timestamp"]
check = nodeApi.verifySignature(data["signature"], toSign)
print("Check signature from blockchain:", check)

# Note, verifySignature isn't unique to Weavechain, it's EdDSA public key cryptography and the verification is done locally

{'signature': '5DJDhMz4GBbqMptFtvMDg3hwYcvEkz2hX1WQz52wVdZuYQ2i8fs3X1U9pxr9XSDn3CnuNNoMvEgZTwWzj7q3JEbw',
 'rootHash': 'EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5',
 'timestamp': '1697703845248'}


Matching: True
Node Public Key: GfHq2tTVk9z4eXgyHRyb7SpxujN2B86QuKWSEvr91g94fVATX72hejve9YcT
Check signature from merkleTree() call: True
Check signature from blockchain: True


### Verify the presence of a record in the merkle tree

In [40]:
import hashlib
import base64
import hmac

#Compute a hash of the data. Use the same salt that is agreed with the server
row = [ 'Nickname', 'Last Name', 'First name', '1980-01-01', 'email@gmail.com', '+40712345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#1234', '@telegram', '0xwallet' ]
recordHash = nodeApi.hashRecord(row, salt, digest)
print("\nData hash:", recordHash)

reply = nodeApi.verifyMerkleHash(session, tree, recordHash, digest).get()
print("Verified to be present in the merkle tree:", reply["data"])


Data hash: 26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb
Verified to be present in the merkle tree: true


### Obtain a merkle proof

In [41]:
reply = nodeApi.merkleProof(session, scope, table, recordHash, digest).get()
print(reply)
data = reply["data"]
display(data)

hashes = [ h.split(",") for h in data["proof"].split(";") ]
display(hashes)

proofSignature = data["signature"]
toSign = data["proof"] + " " + data["timestamp"] # the node signed the proof and the timestamp
check = nodeApi.verifySignature(proofSignature, toSign)
print(check)

{'res': 'ok', 'data': {'signature': '2tTVr3Dg2by8Lp2hUdp7mMbtG6EYcC52hG52nGZwhy9dcApTY3zGg46UWaqViYgTTSbNZgg7rWc836B99F8KK44d', 'rootHash': 'EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5', 'proof': '26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb,4eWYP6nWEH6WVS6Yw7JD3ENgCmiLCfz5YUEPKCTV1PMH;4ZzDmf2gyKgxh9i8kbULc95p8HQSWsxXJh1waq9o4bhy,5JGTsLpou5VTcepwwmvEAR2i6CULnMUPEgwGa3e1hrCg', 'hash': 'SHA-256', 'timestamp': '1697703848218'}}


{'signature': '2tTVr3Dg2by8Lp2hUdp7mMbtG6EYcC52hG52nGZwhy9dcApTY3zGg46UWaqViYgTTSbNZgg7rWc836B99F8KK44d',
 'rootHash': 'EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5',
 'proof': '26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb,4eWYP6nWEH6WVS6Yw7JD3ENgCmiLCfz5YUEPKCTV1PMH;4ZzDmf2gyKgxh9i8kbULc95p8HQSWsxXJh1waq9o4bhy,5JGTsLpou5VTcepwwmvEAR2i6CULnMUPEgwGa3e1hrCg',
 'hash': 'SHA-256',
 'timestamp': '1697703848218'}

[['26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb',
  '4eWYP6nWEH6WVS6Yw7JD3ENgCmiLCfz5YUEPKCTV1PMH'],
 ['4ZzDmf2gyKgxh9i8kbULc95p8HQSWsxXJh1waq9o4bhy',
  '5JGTsLpou5VTcepwwmvEAR2i6CULnMUPEgwGa3e1hrCg']]

True
